In [1]:
import pyodbc, time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By

In [2]:
# DB Definitions:
DB_HOST = '127.0.0.1,41433'
DB_NAME = 'master'
DB_USER = 'sa'
DB_PASSWORD = 'Abc12300'

# BD Methods:

def createConexion(DB_HOST, DB_NAME, DB_USER, DB_PASSWORD):
    connectionString = f'''DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={DB_HOST};DATABASE={DB_NAME};
    UID={DB_USER};PWD={DB_PASSWORD};
    Encrypt=Yes;TrustServerCertificate=Yes'''

    return pyodbc.connect(connectionString)

def dbSelect(conexion, table):
    SQL_QUERY = f"SELECT * FROM {table}"

    #Creamos un cursos para logo acceder aos resultados
    cursor = conexion.cursor()

    #Finalmente executamos a consulta
    cursor.execute(SQL_QUERY)

    tuplas = cursor.fetchall()
    for tupla in tuplas:
        campos = ""
        for campo in tupla:
            campos += f"{campo}\t"
        print(f"{campos}")

    cursor.close()
    conexion.close()

def dbInsertMasivoEmpregado(conexion):
    import traceback

    SQL_PREPARED_STATEMENT = """
        INSERT INTO empregado(nome, apelidos, dni, data_creacion)
        VALUES (?,?,?, CURRENT_TIMESTAMP);
    """

    conexion = pyodbc.connect(conexion)
    cursor = conexion.cursor()

    try:
        for sufijo in range(0, 10):

            cursor.execute(
                SQL_PREPARED_STATEMENT,
                f'nome_{sufijo}',
                f'apelidos_{sufijo}',
                f'{sufijo}2345678A')
            conexion.commit()

    except Exception as e:
        # Print the error and traceback
        print(e)
        traceback.print_exc()
        # Rollback in case of error
        conexion.rollback()
    finally:
        # Close the cursor and connection
        cursor.close()
        conexion.close()

In [ ]:
# Scraping Definitions:
URL="https://contrataciondelestado.es/wps/portal/plataforma"

# Scraping Methods:

def clickRowByName(driver, name, page = None, i = 0):
    if (page == None):
        page = driver

    for row in getElements(driver, By.TAG_NAME, "tr", page=page):
        element = getElement(row, By.CLASS_NAME, "tafelTreecontent")
        if (element.text == name):
            if (i <= 0):
                getElement(row, By.CLASS_NAME, "tafelTreeopenable").click()
                return
            else:
                i-=1
        
def clickLabelByName(driver, name, page = None, i = 0):
    if (page == None):
        page = driver

    for row in getElements(driver, By.TAG_NAME, "tr", page=page):
        element = getElement(row, By.CLASS_NAME, "tafelTreecontent")
        if (element.text == name):
            if (i <= 0):
                element.click()
                return
            else:
                i-=1


def waitUntilPageLoaded(driver, time = 10):
    return WebDriverWait(driver, time).until(
        lambda d: d.execute_script("return document.readyState") == "complete"
    )

def waitUntilElementLoaded(driver, by, identifier, time = 10):
    return WebDriverWait(driver, time).until(
        expected_conditions.presence_of_element_located((by, identifier))
    )


def changeScreen(driver, time = 10):
    waitUntilPageLoaded(driver, time)
    driver.get(driver.execute_script("return window.location.href;"))


def getElement(driver, by, identifier, time = 10):
    waitUntilElementLoaded(driver, by, identifier, time)
    return driver.find_element(by, identifier)

def getElements(driver, by, identifier, page = None, time = 10):
    if (page == None):
        page = driver

    waitUntilPageLoaded(page, time)
    return driver.find_elements(by, identifier)


def generateLinks(driver):
    anhadir = list()
    while(True):
        table = getElement(driver, By.ID, "myTablaBusquedaCustom")
        rows = getElements(table, By.TAG_NAME, "tr", page=driver)

        for row in rows[1:]:
            try:
                expediente = getElement(row, By.CLASS_NAME, "tdExpediente")
                divs = getElements(expediente, By.TAG_NAME, "div", page=driver)
                anhadir.append(
                    getElements(divs[0], By.TAG_NAME, "a", page=driver)[1].get_attribute("href")
                )
            except(Exception):
                # Existen algunas "tr" que tienen formato extraño 
                # así que seguimos la táctica "Let It Crash"
                pass

        elements = driver.find_elements(By.XPATH, "//input[contains(@class, 'botonEnlace') and @title='Siguiente']")
        if (len(elements) <= 0):
            elements[0].click()
            break
    
    return anhadir
            
def calculateLinks(driver, data):
    result = list()
    for link in data:
        driver.get(link)
        nueva_fila = dict()

        nueva_fila["id"] =  getElement(driver, By.ID, "fila0_columna0").find_elements(By.TAG_NAME, "span")[1].text

        datos = getElements(driver, By.CLASS_NAME, "capaAtributos")
        for dato in datos:
            for row in getElements(dato, By.TAG_NAME, "ul", page=driver):
                ils = getElements(row, By.TAG_NAME, "li", page=driver)

                clave = getElement(ils[0],By.TAG_NAME, "span").text.replace("\"", "'")
                valor = getElement(ils[1], By.TAG_NAME, "span").text.replace("\"", "'")

                nueva_fila[clave] = valor

        nueva_fila["id"] =  getElements(
                                getElement(driver, By.ID, "fila0_columna0"),
                                By.TAG_NAME,
                                "span",
                                driver,
                                0
                                )[1].text

        result.append(nueva_fila)
        
    return result


In [ ]:

driver = webdriver.Firefox()
driver.get(URL)

data = None
try:
    # Publicaciones
    getElements(driver, By.CLASS_NAME, "card-item")[0].click()

    # Licitaciones
    getElement(driver, By.XPATH, "/html/body/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div/form/div[1]/div[1]/a/p[2]/span").click()
    
    # Búsqueda avanzada
    getElement(driver, By.XPATH, "//span[@class='rutaGuiada']").click()

    # Seleccionar
    getElement(driver, By.XPATH, "//span[contains(text(), 'Seleccionar')]").click()

    # Englobado de Arbol de Localización
    divs_rows = getElements(driver, By.CLASS_NAME, "tafelTree_row")[2]
    getElement(divs_rows, By.CLASS_NAME, "tafelTreeopenable").click()

    # Arbol de Localización
    clickRowByName(divs_rows, "Galicia", page=driver)
    clickRowByName(divs_rows, "A Coruña", page=driver)
    clickRowByName(divs_rows, "Ayuntamientos", page=driver)
    clickLabelByName(divs_rows, "A Coruña", page=driver, i = 1)

    # Opción Junta de Gobierno del Ayuntamiento de A Coruña
    cuadro = getElement(driver, By.XPATH, "//div//span//select")
    opciones = getElements(cuadro, By.TAG_NAME, "option", page = driver)
    for opcion in opciones:
        if(opcion.get_attribute("value") == "93655277"):
            # Junta de Gobierno del Ayuntamiento de A Coruña
            opcion.click()

    # Botón Añadir
    getElement(driver, By.XPATH, "//input[@value='Añadir']").click()
    
    # Botón Buscar
    getElement(driver, By.XPATH, "//input[@value='Buscar']").click()

    data = generateLinks(driver)

except(Exception) as error:
    print("Error: ", error)

result = None
if (data != None):
    result = calculateLinks(driver, data)
    print("Terminé!")
else:
    print("No existen enlaces")

driver.quit()

The geckodriver version (0.35.0) detected in PATH at C:\Users\gabriel.antonio.edre\mis-binarios\geckodriver.exe might not be compatible with the detected firefox version (136.0.2.571); currently, geckodriver 0.36.0 is recommended for firefox 136.*, so it is advised to delete the driver in PATH and retry


Terminé!


In [7]:
import json

# Escribir el archivo
with open("result.json", "w", encoding="utf-8") as archivo:
    json.dump(result, archivo, indent=4, ensure_ascii=False)

# conexion = createConexion(DB_HOST, DB_NAME, DB_USER, DB_PASSWORD)

# dbSelect(conexion, "empregado")